In [17]:
import datetime
import json

import numpy as np
from itertools import product
import torch
from torch.optim.lr_scheduler import ReduceLROnPlateau

from parametric_pooling_net_ordering import ParametricNetWithPoolingOrdered
from difference_utils import perform_step_ahead_deltas, compute_iteration_rNMSE_with_deltas, \
    visualize_predictions, visualize_deltas
from train_utils import train_model_regression
from evaluation import MSELossWithSparsityRegularizer, rNMSELoss, compute_iteration_rNMSE
from pred_utils import get_device, transform_data_to_all_steps_prediction, get_name_string, get_NOAA_dataset
from misc_utils import check_create_folder
from layers import CPGNN_ST, CPGNN_ST_v2, CPGNN_ST_v3, CITRUS, SGPModel
import networkx as nx
from Utilsss import get_evcs_evals

In [ ]:
torch.cuda.current_device()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)


# torch.manual_seed(123)
# np.random.seed(123)
# random.seed(123)

device = get_device(use_gpu=True)

cuda
Device selected: cuda:0


In [19]:
ds_folder = ""
splits = [0.35, 0.15, 0.5]
obs_window = 10
DIFFERENCE = False

data, steps_ahead, weighted_adjacency = get_NOAA_dataset(
    ds_folder,
    splits=splits,
    obs_window=obs_window,
    differenced=DIFFERENCE,
)
N_spatial_nodes = weighted_adjacency.shape[0]
print(f"{N_spatial_nodes} nodes - {obs_window} observed timesteps - steps ahead: {steps_ahead}")



NOAA is selected



Dataset path: ./dataset/processed/NOA_w=10_steps=[1, 2, 3, 4, 5]_splits=[0.35, 0.15, 0.5].pickle
109 nodes - 10 observed timesteps - steps ahead: [1, 2, 3, 4, 5]


In [20]:
# Get data
trn_data, val_data, tst_data_deltas, trn_labels, val_labels, tst_labels_deltas = transform_data_to_all_steps_prediction(data, node_first=True, device=device)
trn_data = trn_data.float()
val_data = val_data.float()
tst_data_deltas = tst_data_deltas.float()
trn_labels = trn_labels.float()
val_labels = val_labels.float()
tst_labels_deltas = tst_labels_deltas.float()
print(trn_data.shape, val_data.shape, tst_data_deltas.shape)

torch.Size([3051, 1, 109, 10]) torch.Size([1300, 1, 109, 10]) torch.Size([4366, 1, 109, 10])


In [21]:
# obtain one-step labels for the training
one_step_trn_labels = trn_labels[:, 0, :]  # [batch x step-ahead x nodes]
one_step_val_labels = val_labels[:, 0, :]
print(one_step_trn_labels.shape, one_step_val_labels.shape)

torch.Size([3051, 109]) torch.Size([1300, 109])


In [22]:
N_ITERATIONS = 10
num_epochs = 400
learning_rate = 0.01# 0.0005
weight_decay = 0.0025  # , 0.00001, 0]
batch_size = 256  # 32
patience = 100
factor = 0.9

not_learning_limit = 100
lambda_value = 0  # 0.00025, 0.0005, 0.001, 0.005, 0.01, 0.05]

In [23]:
# GTCNN
n_nodes = weighted_adjacency.shape[0]
M = obs_window
N = [n_nodes, M]
K_list = list(np.array(N)-2)
# K_list = [2, 2]

adj = weighted_adjacency
Graph_List = [nx.from_numpy_array(np.array(adj)), nx.path_graph(obs_window)]


In [24]:
evecs, evals, L_list = get_evcs_evals(Graph_List, K_list)

for ii in range(len(evals)):
    evals[ii] = evals[ii].to(device)
    
dim = 4
N_block = 3

today = datetime.datetime.now().strftime("%Y%m%d%H%M%S")

res_dict = {
        'lr': learning_rate,
        'results': []
    }

[107, 8]
evecs.shape:,  torch.Size([109, 107])
evecs.shape:,  torch.Size([10, 8])
evecs_kron.shape:,  torch.Size([10, 8])


In [25]:
for i in range(N_ITERATIONS):
    print(100*'*' + ' iter: ' + str(i) + 100*'*')
    one_step_gtcnn = CITRUS(input_size=1,
                                n_nodes=n_nodes,
                                horizon=1,
                                emb_size=dim,
                                hidden_size=dim,
                                rnn_layers=1,
                                gnn_kernel=1,
                                mass = torch.ones(np.prod(N)).to(device),
                                evals = evals,
                                evecs = torch.tensor(evecs).to(device),
                                C_width = dim,
                                N_block = N_block,
                                single_t = False,
                                use_gdc = [],
                                num_nodes = N,
                                last_activation=torch.nn.LeakyReLU(), 
                                mlp_hidden_dims=[dim, dim, dim, dim], 
                                dropout=False, 
                                with_MLP=True, 
                                diffusion_method='spectral', 
                                device = device,
                                graph_wise=False).to(device)
    one_step_gtcnn.to(device)
    print(one_step_gtcnn)

    model_parameters = filter(lambda p: p.requires_grad, one_step_gtcnn.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    print(f"Number of parameters: {params}")

    log_dir = f"./runs_NOAA_w={obs_window}/{today}_lr={learning_rate}_b={batch_size}_CPGNN_dim={dim}_l={N_block}_eig1={K_list[0]}_eig2={K_list[1]}"

    check_create_folder(log_dir)

    ### TRAINING ###
    # loss_criterion = rNMSELossWithSparsityRegularizer(one_step_gtcnn, lambda_reg)
    loss_criterion = MSELossWithSparsityRegularizer(one_step_gtcnn, lambda_value)

    val_metric = rNMSELoss()

    optimizer = torch.optim.Adam(one_step_gtcnn.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = ReduceLROnPlateau(optimizer, 'min', patience=patience, factor=factor)

    best_model, best_epoch = train_model_regression(Iter=i,
        model=one_step_gtcnn,
        training_data=trn_data, validation_data=val_data,  # [n_samples x 1 x nodes x timesteps]
        single_step_trn_labels=one_step_trn_labels, single_step_val_labels=one_step_val_labels,  # [n_samples x spatial_nodes]
        num_epochs=num_epochs, batch_size=batch_size,
        loss_criterion=loss_criterion, optimizer=optimizer, scheduler=scheduler,
        val_metric_criterion=val_metric,
        log_dir=log_dir,
        not_learning_limit=not_learning_limit
    )

    rNMSE_dict, predictions_dict = compute_iteration_rNMSE(best_model, steps_ahead, tst_data_deltas, tst_labels_deltas,
                                                           device, verbose=False)

    res_dict['results'].append([round(l.item(), 4) for l in list(rNMSE_dict.values())])

    means = [round(el, 4) for el in np.average(res_dict['results'], axis=0)]
    stds = [round(el, 4) for el in np.std(res_dict['results'], axis=0)]
    res_dict['final_res'] = {
        'avg': means,
        'std': stds
    }

    with open(log_dir + '/results.json', 'w', encoding='utf-8') as f:
        json.dump(res_dict, f, ensure_ascii=False, indent=4)

    print(res_dict['results'])

**************************************************************************************************** iter: 0****************************************************************************************************
CITRUS(
  (node_embeddings): NodeEmbedding(n_nodes=109, embedding_size=4)
  (encoder): Linear(in_features=5, out_features=4, bias=True)
  (CPGNN): CPGNN_ST_in_TTS(
    (last_activation): LeakyReLU(negative_slope=0.01)
    (first_lin): Linear(in_features=4, out_features=4, bias=True)
    (last_lin): Linear(in_features=4, out_features=4, bias=True)
    (merge_lin): Linear(in_features=1090, out_features=1, bias=True)
    (node_embeddings): NodeEmbedding(n_nodes=109, embedding_size=4)
    (block_0): CPGNN_block_v2(
      (channel_mixer): Linear(in_features=4, out_features=4, bias=True)
      (diff_derivative): Time_derivative_diffusion_product(
        (Conv_layer): GCN_diff(
          (conv1): GCNConv(4, 4)
        )
      )
      (mlp): MiniMLP(
        (miniMLP_mlp_layer_000): Line

/tmp/ipykernel_8616/2994952299.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  evecs = torch.tensor(evecs).to(device),


Iter 0
 Epoch 0
	 train-loss: 140.436 | valid-loss: 375.205 	| valid-metric: 1.02 | lr: 0.01

				New best val_metric: 1.02. Saving model...

Iter 0
 Epoch 1
	 train-loss: 43.816 | valid-loss: 376.136 	| valid-metric: 1.021 | lr: 0.01
Iter 0
 Epoch 2
	 train-loss: 25.792 | valid-loss: 374.598 	| valid-metric: 1.019 | lr: 0.01

				New best val_metric: 1.019. Saving model...

Iter 0
 Epoch 3
	 train-loss: 22.87 | valid-loss: 369.338 	| valid-metric: 1.012 | lr: 0.01

				New best val_metric: 1.012. Saving model...

Iter 0
 Epoch 4
	 train-loss: 21.156 | valid-loss: 362.148 	| valid-metric: 1.002 | lr: 0.01

				New best val_metric: 1.002. Saving model...

Iter 0
 Epoch 5
	 train-loss: 19.998 | valid-loss: 352.909 	| valid-metric: 0.989 | lr: 0.01

				New best val_metric: 0.989. Saving model...

Iter 0
 Epoch 6
	 train-loss: 18.956 | valid-loss: 340.314 	| valid-metric: 0.971 | lr: 0.01

				New best val_metric: 0.971. Saving model...

Iter 0
 Epoch 7
	 train-loss: 17.505 | valid-loss: 

KeyboardInterrupt: 

In [ ]:
model_parameters = filter(lambda p: p.requires_grad, one_step_gtcnn.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(f"Number of parameters: {params}")


Number of parameters: 2544.0


In [ ]:
# print(res_dict['results'])
means = [round(el, 4) for el in np.average(res_dict['results'], axis=0)]
stds = [round(el, 4) for el in np.std(res_dict['results'], axis=0)]
print('means: ', means)
print('stds: ', stds)

means:  [0.9485, 0.9515, 0.9542, 0.9566, 0.9585]
stds:  [0.0262, 0.027, 0.0272, 0.027, 0.0263]


In [ ]:
res_dict['final_res'] = {
    'avg': means,
    'std': stds
}
with open(log_dir + '/results.json', 'w', encoding='utf-8') as f:
    json.dump(res_dict, f, ensure_ascii=False, indent=4)